# Decision Tree Time Complexity

## 0. Notation and Setup

* Data: $\{(x_i, y_i)\}_{i=1}^n$, $x_i \in \mathbb{R}^p$.
* Tree built by recursive binary splitting.
* “Balanced” means each split roughly halves the samples: $n_L \approx n_R \approx n/2$.
* “Worst case” means maximally unbalanced: $n_L=1,\; n_R=n-1$.
* For classification, let $K$ be the number of classes (treat $K$ as a small constant unless otherwise noted).

**Split core:**
At a node with $n$ samples, searching for the best split across all $p$ features.

---

## 1. Naïve CART (per-node sort)

### 1.1 Per-Node Complexity

**Theorem 1 (Naïve per-node cost).**
Finding the best split at one node costs $T_{\text{split}}(n,p) = O(p\,n\log n)$.

**Proof (step by step).**

1. For each feature $j \in \{1,\dots,p\}$, sort the $n$ values: $O(n \log n)$.
2. Sweep through the sorted list once, maintaining running (left/right) statistics:

   * Regression: constant-time updates per candidate threshold $\Rightarrow O(n)$.
   * Classification: $O(K)$ per threshold with running counts $\Rightarrow O(Kn)$ (take $K$ as constant).
3. Per feature: $O(n\log n) + O(n) = O(n\log n)$.
4. Over $p$ features: $O(p\,n\log n)$. ∎

**Remark (why not $O(p\,n^2)$?).**
The running-sum trick avoids recomputing impurities from scratch; we only do $O(1)$ (or $O(K)$) work per adjacent threshold in sorted order.

---

### 1.2 Whole-Tree Complexity (Naïve)

Let $T(n,p)$ be total time to build a tree on $n$ samples.

**Recurrence (any split):**

$$
T(n,p) \;=\; O(p\,n\log n) \;+\; T(n_L,p) \;+\; T(n_R,p),\quad n_L+n_R=n.
$$

#### Worst case (maximally unbalanced)

$$
T(n,p) \;=\; O(p\,n\log n) \;+\; T(n-1,p).
$$

**Unroll (explicit):**

$$
T(n,p) = \sum_{k=1}^{n} O(p\,k\log k) 
= O\!\Big(p\sum_{k=1}^{n} k\log k\Big).
$$

**Bound the sum (integral comparison):**

$$
\sum_{k=1}^{n} k\log k 
\;\;=\;\; \Theta(n^2\log n).
$$

Therefore $T(n,p)=O(p\,n^2\log n)$.

**ASCII view (worst case):**

```
Level 0:  n samples  → cost ~ c p n log n
Level 1:  n-1        → cost ~ c p (n-1) log(n-1)
Level 2:  n-2        → cost ~ c p (n-2) log(n-2)
...
Level n-1: 1         → cost ~ c p (1) log 1 ≈ 0
Sum ~ c p * [n log n + (n-1)log(n-1) + ...] = O(p n^2 log n)
```

#### Balanced case

$$
T(n,p) \;=\; 2T\!\left(\frac{n}{2},p\right) + O(p\,n\log n).
$$

**Recursion tree (work per level):**

* Level $\ell$: $2^\ell$ nodes, each of size $\approx n/2^\ell$.
* Per node cost: $c\,p\,(n/2^\ell)\log(n/2^\ell)$.
* Level $\ell$ total:

  $$
  2^\ell \cdot c\,p\,(n/2^\ell)\big(\log n - \ell\big)
  \;=\; c\,p\,n\big(\log n - \ell\big).
  $$
* Depth: $\log_2 n$.

**Sum over levels $\ell=0,\dots,\log n - 1$:**

$$
\sum_{\ell=0}^{\log n - 1} c\,p\,n(\log n - \ell)
= c\,p\,n \underbrace{\sum_{t=1}^{\log n} t}_{=\;\Theta((\log n)^2)}
= \Theta(p\,n\,(\log n)^2).
$$

**ASCII recursion tree (balanced, naïve):**

```
Level 0: 1 node of size n        → cost ≈ c p n (log n)
Level 1: 2 nodes of size n/2     → cost ≈ 2 * c p (n/2) (log(n/2)) = c p n (log n - 1)
Level 2: 4 nodes of size n/4     → cost ≈ c p n (log n - 2)
...
Level L: 2^L nodes size n/2^L    → cost ≈ c p n (log n - L)

Sum L=0..log n-1 of [c p n (log n - L)] = c p n [log n + (log n - 1) + ... + 1]
= Θ(p n (log n)^2)
```

**Conclusion (naïve CART):**

* Worst case: $O(p\,n^2\log n)$
* Balanced: $O(p\,n\log^2 n)$

---

## 2. Presorted CART (classic optimization)

**Idea.** Presort each feature **once** at the root and maintain **sorted index lists** down the tree. Then each node no longer pays $O(n\log n)$ per feature—only a **linear sweep** and an $O(n)$ partition of indices per child.

### 2.1 Per-Node Complexity (Presorted)

**Theorem 2 (Presorted per-node cost).**
With per-feature presorting carried once and index lists maintained, the per-node split search costs $O(p\,n)$ (plus $O(n)$ to partition indices), not $O(p\,n\log n)$.

**Proof (sketch).**

* At a node, for each feature, we already have that node’s samples in **sorted order** (by inheriting/partitioning the parent’s sorted lists).
* We sweep once per feature, maintaining cumulative statistics $\Rightarrow O(n)$ per feature $\Rightarrow O(p\,n)$.
* Partitioning the sorted index lists into left/right for the chosen split is $O(n)$ per feature **in total**, but with careful bookkeeping you only physically partition **once** for the chosen split; the other features’ index orders are **filtered** to children via stable partition or pointer views. Practical implementations make this linear in $n$. ∎

### 2.2 Whole-Tree Complexity (Presorted)

Balanced recurrence:

$$
T(n,p) \;=\; 2T\Big(\frac{n}{2},p\Big) + O(p\,n).
$$

By Master Theorem ($a=2, b=2, f(n)=\Theta(n)$):

$$
T(n,p) \;=\; O(p\,n\log n).
$$

Worst case (unbalanced):

$$
T(n,p) \;=\; O(p\,n) + T(n-1,p) \;\Rightarrow\; O(p\,n^2).
$$

**ASCII recursion tree (balanced, presorted):**

```
Level 0:  cost ≈ c p n
Level 1:  cost ≈ c p n
...
Level log n - 1: cost ≈ c p n
#levels ≈ log n → total ≈ c p n log n
```

**Takeaway:** Presorting removes one $\log n$ factor vs naïve CART.

---

## 3. Histogram CART (binning)

**Key idea.** Discretize each feature into $B \ll n$ bins, and evaluate splits only at **bin boundaries**.

### 3.1 Two variants you can teach

* **Variant A (simple):** Per node, reassign all $n$ samples into bins for all $p$ features $\Rightarrow O(p\,n)$ per node; then evaluate $B-1$ thresholds $\Rightarrow O(p\,B)$.
* **Variant B (incremental/prefix sums):** Maintain **prefix sums** of bin statistics and update **incrementally** when splitting, so the dominant node cost is $O(p\,B)$ (nearly independent of $n$).

### 3.2 Preprocessing

* Build bin edges per feature (quantiles or uniform): $O(p\,n\log n)$ if using quantiles; $O(p\,n)$ if using single pass with approximate quantiles / sketches.
* Initialize per-feature, per-bin statistics.

### 3.3 Per-Node Costs (detailed)

**Theorem 3 (Histogram per-node cost).**

* Variant A: $O(p\,n) + O(p\,B) = O(p(n+B))$.
* Variant B (incremental): $O(p\,B)$.

**Proof (step by step).**

1. **Assignment / stats.**

   * Variant A: sweep samples $\Rightarrow O(p\,n)$.
   * Variant B: children’s histograms = parent’s prefix sums “cut” at the chosen threshold; updates are $O(p\,B)$.
2. **Split search.**
   For each feature, try $B-1$ bin cuts; with running prefix/suffix sums the scan is $O(B)$ per feature $\Rightarrow O(p\,B)$.
3. Sum both parts. ∎

### 3.4 Whole-Tree Complexity (balanced)

* **Variant A (simple, with re-binning):**

  $$
  T(n,p) = 2T(n/2,p) + O(p\,n) \;\Rightarrow\; O(p\,n\log n).
  $$
* **Variant B (incremental, $O(pB)$ per node):**
  There are $O(n)$ total nodes across all levels (bounded by $2n\!-\!1$ in a binary tree with $n$ leaves).
  Total $= O\big((\#\text{nodes}) \cdot pB\big) = O(p\,B\,n)$.
  Add preprocessing $O(p\,n\log n)$ (quantile bins).
  **Total:** $O(p\,n\log n + p\,B\,n)$.
  For fixed $B$ (e.g., $B\le 255$), this is $O(p\,n\log n)$.

**ASCII recursion tree (balanced, Variant A):**

```
Level 0:  cost ≈ c p n
Level 1:  cost ≈ c p n
...
Level log n - 1: cost ≈ c p n
Total ≈ c p n log n
```

**Worst case (unbalanced).**

* Variant A: $\sum_{k=1}^{n} O(p\,k) = O(p\,n^2)$.
* Variant B: $\sum_{k=1}^{n} O(p\,B) = O(p\,B\,n)$ (plus preprocessing).

**Practical note.** Modern GBM libraries approximate quantiles with sketches to reduce preprocessing to near-linear time and memory.

---

## 4. Worked Mini-Examples

### 4.1 Balanced, naïve CART with $n=8$, $p$ fixed

Costs per level:

* Level 0: $c\,p\,8\log 8 = c\,p\,8\cdot 3 = 24cp$
* Level 1: $2 \times c\,p\,(4\log 4) = 2 \times c\,p\,4\cdot 2 = 16cp$
* Level 2: $4 \times c\,p\,(2\log 2) = 4 \times c\,p\,2\cdot 1 = 8cp$

Total $= (24+16+8)cp = 48cp = c\,p\,8\cdot (3+2+1) = c\,p\,n \sum_{t=1}^{\log n} t$.

### 4.2 Balanced, presorted CART with $n=8$

Each level costs $\approx c\,p\,n = 8cp$, and there are $\log_2 8=3$ levels.
Total $= 3 \cdot 8cp = 24cp = O(p\,n\log n)$.

### 4.3 Balanced, histogram (Variant A) with $n=8$

Each level costs $\approx c\,p\,n = 8cp$, three levels $\Rightarrow 24cp$, same asymptotics as presorted: $O(p\,n\log n)$.
If Variant B is used, each node costs $\approx c\,p\,B$, total nodes scale $O(n)$ $\Rightarrow O(p\,B\,n)$ plus preprocessing.

---

## 5. Memory & Cache, Step-by-Step

### 5.1 Memory

* **Naïve / Presorted CART**

  * Data matrix: $O(n\,p)$.
  * Presorted indices: $O(n\,p)$ integers (can be kept per feature, shared across nodes via views).
  * Tree: $O(\#\text{nodes}) \le O(n)$.

* **Histogram CART**

  * Bin edges: $O(p\,B)$.
  * Bin statistics: classification $O(p\,B\,K)$, regression $O(p\,B)$.
  * Tree: $O(n)$.

### 5.2 Cache

* **Naïve (per-feature scans over $n$ values)** → many cache misses for large $n$.
* **Histogram (operate on $B$ counters)** → $B\ll n$ fits in L2/L3; sequential memory patterns; drastically fewer misses.

---

## 6. Parallel Complexity (outline)

* **Naïve / Presorted CART**

  * Per-feature split scans can parallelize across features: up to $p$-way parallelism (memory-bandwidth limited).
  * Across nodes at the same depth: also parallelizable.

* **Histogram CART**

  * Building/merging histograms across features and nodes is **embarrassingly parallel**.
  * Variant B (prefix sums) benefits from SIMD-friendly scans over small $B$.

---

## 7. Ensemble Costs (per tree contrasted)

| Method             | Per-node cost   | Balanced total per tree      | Worst-case total per tree |
| ------------------ | --------------- | ---------------------------- | ------------------------- |
| Naïve CART         | $O(p\,n\log n)$ | $O(p\,n\log^2 n)$            | $O(p\,n^2\log n)$         |
| Presorted CART     | $O(p\,n)$       | $O(p\,n\log n)$              | $O(p\,n^2)$               |
| Histogram (Var. A) | $O(p(n+B))$     | $O(p\,n\log n)$              | $O(p\,n^2)$               |
| Histogram (Var. B) | $O(p\,B)$       | $O(p\,n\log n) + O(p\,B\,n)$ | $O(p\,B\,n)$              |

**Random Forest (B trees).**

* Using presorted or histogram trees:

  * Presorted, balanced: $O(B\,p\,n\log n)$.
  * Histogram (Var. B), balanced: $O(B\,(p\,n\log n + p\,B\,n))$.
* **Parallelism:** nearly linear across trees.

**Gradient Boosting (M iterations).**

* Each iteration fits one tree to residuals (same per-tree cost as above).
* Total:

  * Presorted: $O(M\,p\,n\log n)$ (balanced).
  * Histogram (Var. B): $O(M\,(p\,n\log n + p\,B\,n))$.
* **Sequential bottleneck** across iterations persists.

---

## 8. Final Comparison (intuitive summary)

1. **Naïve CART** pays $\log n$ inside each level → $O(p\,n\log^2 n)$ balanced.
2. **Presorted CART** removes that inner $\log n$ → $O(p\,n\log n)$ balanced.
3. **Histogram CART** trades exact thresholds for $B$ bins:

   * Simple Variant A re-bins: $O(p\,n\log n)$ balanced.
   * Incremental Variant B makes per-node work \~$O(p\,B)$ → total $O(p\,n\log n + p\,B\,n)$; with fixed small $B$, this is effectively $O(p\,n\log n)$ and very cache-friendly.

---

## 9. ASCII Recursion Trees — Ready for Slides

### 9.1 Naïve CART (balanced)

```
                   n (cost ~ c p n log n)
                   /                     \
        n/2 (c p (n/2) log(n/2))   n/2 (c p (n/2) log(n/2))
           /           \               /           \
      n/4 (...)    n/4 (...)      n/4 (...)    n/4 (...)
        .                                 .
        .                                 .
Depth ~ log2 n

Work per level:
  L = 0:  c p n (log n)
  L = 1:  c p n (log n - 1)
  ...
  L = log n - 1: c p n (1)

Total ≈ c p n * (1 + 2 + ... + log n) = Θ(p n (log n)^2)
```

### 9.2 Presorted CART (balanced)

```
                   n (cost ~ c p n)
                   /               \
            n/2 (c p n/2)      n/2 (c p n/2)
               /      \           /      \
           n/4        n/4     n/4        n/4
               ...                 ...

Work per level:
  Always ≈ c p n
#levels ≈ log n → total ≈ c p n log n
```

### 9.3 Histogram CART (Variant A; re-bin each node)

```
Same shape as presorted.
Work per level still ≈ c p n (for the re-binning)
Total ≈ c p n log n
```

### 9.4 Histogram CART (Variant B; incremental O(pB) per node)

```
Every node ~ O(p B), independent of its n_i
Total nodes in a binary tree with n leaves: O(n)
Total ≈ O(p B n)  (+ preprocessing O(p n log n) for quantile bins)
```

---

## 10. Pedagogical Takeaways (for students)

* **One optimization removes one log.** Naïve vs presorted is the difference between $\log^2 n$ and $\log n$.
* **Histograms buy speed via approximation.** With small $B$, per-node work is tiny and cache-friendly.
* **Balanced vs worst-case matters.** Always mention both: $O(p\,n\log n)$ vs $O(p\,n^2)$ (presorted/hist).
* **Ensembles scale differently.** Random forests parallelize across trees; boosting remains sequential across rounds.
* **Asymptotics vs constants.** For small $n$, constants and cache dominate; for large $n$, these complexity differences decide feasibility.

---

# CumSum Trick

## Setup

Suppose at a node we consider feature $j$. After sorting the samples by $x_{\cdot,j}$, the paired (feature, target) order is:

$$
\big((x_{(1),j},y_{(1)}),\dots,(x_{(6),j},y_{(6)})\big)
\quad\text{with}\quad
y_{(1..6)} = [2,\;3,\;5,\;1,\;4,\;2].
$$

Totals over all 6 samples:

$$
n=6,\quad
S_1^{\text{tot}}=\sum y_i=17,\quad
S_2^{\text{tot}}=\sum y_i^2=59.
$$

For a subset $S$ with size $n_S$, sum $S_1=\sum_{i\in S} y_i$, sum of squares $S_2=\sum_{i\in S} y_i^2$,
the **SSE** is

$$
\mathrm{SSE}(S)=S_2-\frac{S_1^2}{n_S}.
$$

(Weighted MSE impurity is just $(\mathrm{SSE}_L+\mathrm{SSE}_R)/n$; minimizing total SSE or weighted MSE is equivalent for a fixed $n$.)

We consider thresholds **between** consecutive sorted points: after 1st, 2nd, …, 5th sample.

---

## Running-sum sweep (one pass)

Initialize:

* Left (L): $n_L=0,\; S_{1L}=0,\; S_{2L}=0$.
* Right (R): $n_R=6,\; S_{1R}=17,\; S_{2R}=59$.

At step $k$ we move $y_{(k)}$ from R to L and evaluate the split **between** $x_{(k),j}$ and $x_{(k+1),j}$.

Update rules (O(1) each step):

$$
\begin{aligned}
&n_L \leftarrow n_L+1,\quad S_{1L} \leftarrow S_{1L}+y_{(k)},\quad S_{2L} \leftarrow S_{2L}+y_{(k)}^2,\\
&n_R \leftarrow n_R-1,\quad S_{1R} \leftarrow S_{1R}-y_{(k)},\quad S_{2R} \leftarrow S_{2R}-y_{(k)}^2.
\end{aligned}
$$

Then compute

$$
\mathrm{SSE}_L=S_{2L}-\frac{S_{1L}^2}{n_L},\qquad
\mathrm{SSE}_R=S_{2R}-\frac{S_{1R}^2}{n_R},\qquad
\mathrm{SSE}_{\text{total}}=\mathrm{SSE}_L+\mathrm{SSE}_R.
$$

We’ll tabulate all 5 candidate splits:

|  k | move $y_{(k)}$ | $n_L,S_{1L},S_{2L}$ | $n_R,S_{1R},S_{2R}$ |     $\mathrm{SSE}_L$    |    $\mathrm{SSE}_R$    | $\mathrm{SSE}_{\text{total}}$ |
| -: | :------------- | :------------------ | :------------------ | :---------------------: | :--------------------: | :---------------------------: |
|  1 | 2              | $1,2,4$             | $5,15,55$           |        $4-4/1=0$        |  $55-15^2/5=55-45=10$  |           **10.000**          |
|  2 | 3              | $2,5,13$            | $4,12,46$           |      $13-25/2=0.5$      |  $46-12^2/4=46-36=10$  |           **10.500**          |
|  3 | 5              | $3,10,38$           | $3,7,21$            | $38-100/3\approx4.6667$ | $21-49/3\approx4.6667$ |           **9.3333**          |
|  4 | 1              | $4,11,39$           | $2,6,20$            |     $39-121/4=8.75$     |       $20-36/2=2$      |           **10.750**          |
|  5 | 4              | $5,15,55$           | $1,2,4$             |      $55-225/5=10$      |        $4-4/1=0$       |           **10.000**          |

* **Best split** is after $k=3$ (between the 3rd and 4th sorted samples), with
  $\mathrm{SSE}_{\text{total}}\approx 9.3333$.

If you prefer **MSE** impurity (weighted by child sizes), divide the total SSE by $n=6$; the argmin is unchanged.

---

## Why this is fast

* We sorted the feature **once**: $O(n\log n)$.
* Then we evaluated all $n-1$ thresholds in **one linear sweep** with **O(1)** work per threshold.
* **Per feature cost:** $O(n\log n) + O(n) = O(n\log n)$, instead of $O(n^2)$ if we recomputed sums from scratch at each threshold.